<a href="https://colab.research.google.com/github/elainedias16/TCC/blob/main/Copy_of_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Large language models(LLM)

## Motivação

Em 2017, a publicação do artigo "Attention is All You Need" revolucionou o campo do Processamento de Linguagem Natural (NLP) com a introdução dos Transformers. Essa arquitetura neural, com sua capacidade de processar sequências de dados de forma mais eficiente, impulsionou o desenvolvimento de modelos de linguagem cada vez mais sofisticados e poderosos. A partir daí, os Grandes Modelos de Linguagem (LLMs) experimentaram um vasto crescimento, atraindo investimentos consideráveis e abrindo novas fronteiras para a área. Para se manter atualizado nesse cenário, é fundamental compreender o funcionamento dos LLM, de forma a aprimorar e desenvolver novas aplicações.

## Resultados Esperados



Neste laboratório, espera-se que os alunos compreendam os princípios básicos do funcionamento de um Grande Modelo de Linguagem. Para exemplificar, será apresentado um código em pequena escala de um LLM.

## Fundamentação teórica

## Código

In [38]:
pip install transformers torch

## Masked Self-Attention

Scale dot produt attetion:
https://paperswithcode.com/method/scaled

In [39]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class Head(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.query = nn.Linear(config.d_model, config.head_dim, bias=config.bias)
    self.key = nn.Linear(config.d_model, config.head_dim, bias=config.bias)
    self.value = nn.Linear(config.d_model, config.head_dim, bias=config.bias)


  def forward(self, x):
    q = self.query(x)
    k = self.key(x)
    v = self.value(x)
    return q, k, v



class MaskedSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_heads = config.num_heads
        self.head_dim = config.head_dim
        self.dropout = nn.Dropout(config.dropout)
        self.d_model = config.d_model
        self.heads = nn.ModuleList([Head(config) for _ in range(config.num_heads)])
        self.output_linear = nn.Linear(config.d_model, config.d_model)
        assert self.head_dim * self.num_heads == self.d_model, "d_model must be divisible by num_heads"


    def forward(self, x, mask=None):
        # B, T, C = x.size()

        heads_output = []
        for head in self.heads:
            k, q, v = head(x)


            # Scaled dot-product attention
            scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)

            if mask is not None:
                scores = scores.masked_fill(mask == 0, float('-inf'))

            attn_weights = F.softmax(scores, dim=-1)
            attn_weights = self.dropout(attn_weights)

            head_output = torch.matmul(attn_weights, v)
            heads_output.append(head_output)


        concatenated_output = torch.cat(heads_output, dim=-1)
        output = self.output_linear(concatenated_output)

        return output




## Feed Forward Nerual Network


MLP

In [40]:
class FeedFoward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.linear1 = nn.Linear(config.d_model, 4 * config.d_model, bias=config.bias)
    self.activation = nn.ReLU()
    self.linear2 = nn.Linear(config.d_model * 4,  config.d_model, bias=config.bias)
    self.dropout = nn.Dropout(config.dropout)


  def forward(self, x):
    x = self.linear1(x)
    x = self.activation(x)
    x = self.linear2(x)
    x = self.dropout(x)
    return x



## Layer Norm

In [41]:
class LayerNorm(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.norm = nn.LayerNorm(config.d_model, config.bias)

  def forward(self, x):
    self.norm(x)
    return x

## One Decoder

In [42]:
class Decoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.ln_1 = LayerNorm(config)
    self.masked_self_attention = MaskedSelfAttention(config)
    self.ln_2 = LayerNorm(config)
    self.feed_forward = FeedFoward(config)

  # def forward(self, x, mask):
  #   x = self.ln_1(x)
  #   x = x + self.masked_self_attention(x, mask)
  #   x = self.ln_2(x)
  #   x = x + self.feed_forward(x)
  #   return x
  def forward(self, x):
    x = self.ln_1(x)
    x = x + self.masked_self_attention(x)
    x = self.ln_2(x)
    x = x + self.feed_forward(x)
    return x



## Transformer

https://medium.com/@hunter-j-phillips/positional-encoding-7a93db4109e6#:~:text=class%20PositionalEncoding(nn,self.dropout(x)
olhar posiitional

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset

class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.word_token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.position_embedding = nn.Embedding(config.max_length, config.d_model)
        self.dropout = nn.Dropout(config.dropout)
        self.blocks = nn.Sequential(*[Decoder(config) for _ in range(config.n_layer)])
        self.ln = LayerNorm(config)
        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)

    def forward(self, input_ids, targets=None):
        device = input_ids.device
        B, T = input_ids.size()

        # Positional e token embed
        tok_emb = self.word_token_embedding(input_ids)
        pos_emb = self.position_embedding(torch.arange(T, device=device))
        x = self.dropout(tok_emb + pos_emb)

        # Transformer blocks
        x = self.blocks(x)

        # Norm layer
        x = self.ln(x)

        # Final layer
        logits = self.lm_head(x)

        if targets is not None:
            shift_logits = logits[:, :-1, :].contiguous()
            shift_targets = targets[:, 1:].contiguous()
            loss = F.cross_entropy(shift_logits.view(-1, shift_logits.size(-1)), shift_targets.view(-1))
            return logits, loss

        return logits, None

    def generate(self, input_ids, max_new_tokens):
        new_tokens = []

        for _ in range(0, max_new_tokens):
            input_ids_cond = input_ids[:, -self.config.block_size:]
            logits, _ = self.forward(input_ids_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            input_ids_next = torch.multinomial(probs, num_samples=1)
            new_tokens.append(input_ids_next)
            input_ids = torch.cat((input_ids, input_ids_next), dim=1)

        new_tokens = torch.cat(new_tokens, dim=1)
        return new_tokens


## Config

In [44]:
import torch

#Paramters:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_new_tokens = 50
epochs = 100
learning_rate = 0.001
batch_size = 8
SEQUENCE_LENGTH = 64


class Config:
    num_heads = 2
    d_model = 64 #os vetores de entrada e saída terão dimensão 8
    head_dim = 32 #cada cabeça tem dimensão 4
    dropout = 0.1  #para evitar overfiting
    bias = True
    vocab_size = 50257  # len tokenizer
    # hidden_size = 1024
    max_length = 512
    n_layer = 6
    # block_size = 1024
    # block_size = 32
    block_size = SEQUENCE_LENGTH


config = Config()

## Training

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Dataset Preparation

In [46]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

# with open('input.txt', 'r', encoding='utf-8') as f:
#   text = f.read()

In [47]:
!wget https://raw.githubusercontent.com/elainedias16/TCC/main/alice_1.txt

with open('alice_1.txt', 'r', encoding='utf-8') as f:
    text = f.read()

--2024-09-10 19:21:14--  https://raw.githubusercontent.com/elainedias16/TCC/main/alice_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4563 (4.5K) [text/plain]
Saving to: ‘alice_1.txt.3’

alice_1.txt.3       100%[===================>]   4.46K  --.-KB/s    in 0s      

2024-09-10 19:21:14 (51.1 MB/s) - ‘alice_1.txt.3’ saved [4563/4563]



In [48]:
class TextDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [49]:
def tokenize(text):
  return word_tokenize(text.lower())


def build_vocab(tokens):
  # tokens = tokenize(text)
  word_counts = Counter(tokens)
  vocab = list(word_counts.keys())
  word_to_int = {word: i for i, word in enumerate(vocab)}
  int_to_word = {i: word for word, i in word_to_int.items()}

  return len(vocab), word_to_int, int_to_word



def prepare_data(tokens, word_to_int, sequence_length=64):
  # tokens = tokenize(text)
  encoded = [word_to_int[token] for token in tokens]
  samples = [encoded[i:i + sequence_length + 1] for i in range(len(encoded) - sequence_length)]
  input_ids = torch.tensor([sample[:-1] for sample in samples], dtype=torch.long)
  targets = torch.tensor([sample[1:] for sample in samples], dtype=torch.long)
  return input_ids, targets



def tokenize_and_encode(text, word_to_int):
    tokens = tokenize(text)
    if '<pad>' not in word_to_int:
        pad_id = 0
    else:
        pad_id = word_to_int['<pad>']
    encoded = [word_to_int.get(token, pad_id) for token in tokens]
    return torch.tensor(encoded, dtype=torch.long).unsqueeze(0)


# def decode_tokens(tokens, int_to_word):
#     return ' '.join([int_to_word[token.item()] for token in tokens.squeeze()])

def decode_tokens(tokens, int_to_word):
    # Create a list to store the decoded words
    decoded_words = []
    # Iterate over the tokens
    for token in tokens.squeeze():
        # Check if the token ID exists in the int_to_word dictionary
        if token.item() in int_to_word:
            # If it exists, append the corresponding word to the decoded_words list
            decoded_words.append(int_to_word[token.item()])
        else:
            # If not, append a special token (e.g., "<UNK>") to indicate an unknown word
            decoded_words.append("<UNK>")
    # Join the decoded words with spaces and return the result
    return ' '.join(decoded_words)

# def generate_text(model, start_text, word_to_int, int_to_word, max_new_tokens=50):
def generate_text(start_text, word_to_int, int_to_word, max_new_tokens=50):
    model.eval()
    with torch.no_grad():
        input_ids = tokenize_and_encode(start_text, word_to_int)
        generated_tokens = model.generate(input_ids, max_new_tokens)
        return decode_tokens(generated_tokens, int_to_word)


# tokenize_text = tokenize(text)
# vocab_size, word_to_int, int_to_word = build_vocab(tokenize_text)

##Train

## Run

In [50]:
model = Transformer(config)
optimizer = Adam(model.parameters(), lr=learning_rate)

In [56]:
# with open('alice_1.txt', 'r', encoding='utf-8') as file:
#     text = file.read()

# def train_model(model, dataloader, optimizer, epochs):
#     model.train()
#     for epoch in range(epochs):
#         total_loss = 0
#         for input_ids, targets in dataloader:
#             optimizer.zero_grad()
#             logits, loss = model(input_ids, targets)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#         print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")



def train_model(model, dataloader, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for input_ids, targets in dataloader:
            optimizer.zero_grad()
            logits, loss = model(input_ids, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")

In [57]:
text = "Sofia era uma menina quieta e introspectiva, mas havia algo nela que todos sabiam: seu amor pelos livros. Desde muito pequena, tinha uma curiosidade insaciável pelo mundo ao seu redor. Aos cinco anos, já estava folheando livros com ilustrações, fascinada pelas imagens e pelas poucas palavras que conhecia. À medida que crescia, essa paixão só aumentava. Seus pais a encontravam, todos os dias, escondida em algum canto da casa com um livro nas mãos, como se estivesse em outro mundo. Na escola, Sofia não era a criança mais extrovertida, mas os livros lhe davam confiança. Enquanto os colegas brincavam no pátio, ela preferia a biblioteca. A bibliotecária, Dona Clara, rapidamente se tornou sua amiga e confidente. Dona Clara sabia exatamente quais livros indicar para cada fase de Sofia. Desde contos de fadas clássicos até aventuras fantásticas, cada novo livro era uma porta para um mundo cheio de magia e descobertas. Sofia gostava de se imaginar como as protagonistas das histórias que lia. Às vezes, ela era uma exploradora destemida, em outras, uma princesa corajosa que lutava contra o mal. Mas o que mais a fascinava eram as palavras. Ela não apenas lia, mas sentia cada frase, cada parágrafo como se fosse parte da sua própria vida. Isso a inspirou a começar a escrever suas próprias histórias. No começo, suas histórias eram simples, contos sobre princesas, dragões e reinos distantes. No entanto, à medida que crescia, seus escritos se tornaram mais profundos. Ela começou a explorar temas sobre amizade, coragem e superação. Aos 12 anos, já tinha um caderno cheio de histórias que criava, e sonhava, um dia, em publicar seus próprios livros. Para Sofia, ler não era apenas um hobby; era a chave que a conectava a um mundo infinito de imaginação e conhecimento."

In [58]:
tokenize_text = tokenize(text)
vocab_size, word_to_int, int_to_word = build_vocab(tokenize_text)

config.vocab_size, word_to_int, int_to_word = build_vocab(tokenize_text)
input_ids, targets = prepare_data(tokenize_text, word_to_int)


dataset = TextDataset(input_ids, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [60]:
# train_model(model, dataloader, optimizer, epochs=epochs)
train_model(model, dataloader, optimizer, epochs=5)

test_sentences = [
    "Sofia era uma menina quieta",
    "Na escola, Sofia não era a criança mais extrovertida,",
]

for sentence in test_sentences:
    # generated_text = generate_text(model, sentence, word_to_int, int_to_word, max_new_tokens=50)
    generated_text = generate_text(sentence, word_to_int, int_to_word, max_new_tokens=50)
    print(f"Input: {sentence}")
    print(f"Generated: {generated_text}")
    print("="*80)


Epoch 1, Loss: 1.6512410044670105
Epoch 2, Loss: 0.995408241947492
Epoch 3, Loss: 0.76738208035628
Epoch 4, Loss: 0.6815009497933917
Epoch 5, Loss: 0.6237838980224397
Input: Sofia era uma menina quieta
Generated: introspectiva mas algo que . cinco , paixão aumentava seus a temas amizade , paixão aumentava seus livros ilustrações fascinada imagens pelas palavras conhecia à que . medida crescia essa só . cinco , tinha caderno de todos dias escondida algum da com livro mãos como estivesse outro . medida
Input: Na escola, Sofia não era a criança mais extrovertida,
Generated: não a mais . escola sofia era chave a que a temas amizade coragem superação aos anos já um cheio histórias criava e . medida crescia seus se mais . não <UNK> em seus se mais . sofia ler era porta um cheio histórias criava e , dia em seus


In [54]:
train_model(model, dataloader, optimizer, epochs=1)

test_sentences = [
    "Alice was a curious and imaginative young girl",
    "As Alice fell through the rabbit hole,",
]

for sentence in test_sentences:
    # generated_text = generate_text(model, sentence, word_to_int, int_to_word, max_new_tokens=50)
    generated_text = generate_text(sentence, word_to_int, int_to_word, max_new_tokens=50)
    print(f"Input: {sentence}")
    print(f"Generated: {generated_text}")
    print("="*80)

Epoch 1, Loss: 4.973833259910044
Input: Alice was a curious and imaginative young girl
Generated: eccentric , time to escape unknown fist the hatter accompanied . mischievous continue distort dare the hatter accompanied shrink all the hatter accompanied with wise turmoil help returning , noticed into us with of flowers each declared 's palace adventures '' . small uncover secrets off lingering wonderland in of
Input: As Alice fell through the rabbit hole,
Generated: decided as she seemed distort to the rabbit the around a dream yet she us grinning had distort she of sizes to surprise she . frustrated . , nearby me familiar , decided a rabbit a been rabbit the she to appeared alice when a , took small fell she


# Referências

Architeture
https://dugas.ch/artificial_curiosity/GPT_architecture.html

https://keras.io/examples/generative/text_generation_with_miniature_gpt/

https://huggingface.co/learn/nlp-course/chapter7/6

https://debuggercafe.com/text-generation-with-transformers/0

https://debuggercafe.com/text-generation-with-transformers/


https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf